# Notebook 1:  The FHIR API

## Learning the FHIR REST API

In [ ]:
# setup imports
import os
from requests import get
from requests import post
from requests import put
from requests import delete
from requests import head

# save the base url of the server
base = 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open'

# define some useful functions
def peek(string, line_count=10):
    print(os.linesep.join(string.split(os.linesep)[:line_count]) + '\n')

In [ ]:
#(Optional)

# install jsonpointer
!pip install jsonpointer

from jsonpointer import resolve_pointer as resolve

In [50]:
# retrieve the server "CapabilityStatement" and print the important bits
response = get(base + '/metadata')
print('Response code: ' + str(response.status_code))
result = response.json()
print('Server: ' + result['name'] + ' ' + result['version'])
print('Security: ' + str(result['rest'][0]['security']))
resources = result['rest'][0]['resource']

supported_types = {r['type']: [i['code'] for i in r['interaction']] for r in resources}

print('Supported types: ')
for k,v in supported_types.items():
    print('  ' + k + ': ' + str(v))

Response code: 200
Server: IBM FHIR Server 4.5.0
Security: {'cors': True}
Supported types: 
  Endpoint: ['read', 'vread', 'history-instance', 'search-type']
  Goal: ['read', 'vread', 'history-instance', 'search-type']
  Library: ['read', 'vread', 'history-instance', 'search-type']
  Contract: ['read', 'vread', 'history-instance', 'search-type']
  Flag: ['read', 'vread', 'history-instance', 'search-type']
  CodeSystem: ['read', 'vread', 'history-instance', 'search-type']
  CapabilityStatement: ['read', 'vread', 'history-instance', 'search-type']
  List: ['read', 'vread', 'history-instance', 'search-type']
  ChargeItem: ['read', 'vread', 'history-instance', 'search-type']
  PractitionerRole: ['read', 'vread', 'history-instance', 'search-type']
  Location: ['read', 'vread', 'history-instance', 'search-type']
  MedicinalProductIndication: ['read', 'vread', 'history-instance', 'search-type']
  ResearchElementDefinition: ['read', 'vread', 'history-instance', 'search-type']
  MedicinalProduct

In [51]:
# the type of resource we want to query
type = 'Patient'

# retrieve all resources of a given type and print the HTTP status code and the first 20 lines of the response
response = get(base + '/' + type)
print('Response code: ' + str(response.status_code))
peek('Response body: \n' + response.text, 20)

# technically you've now performed your first FHIR "search" (just with no parameters)

Response code: 200
Response body: 
{
    "resourceType": "Bundle",
    "id": "9d7424fe-7235-4e1a-8c6d-058d2603b263",
    "type": "searchset",
    "total": 27319,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
            "resource": {



In [52]:
# results are paged and the "link" field in the response Bundle contains links to previous, current, and next page of results
for link in response.json().get('link'):
    if link.get('relation') == 'next':
        page2 = get(link.get('url'))        
peek('Second page: \n' + page2.text, 20)
print('Number of entries: ' + str(len(response.json().get('entry'))))

Second page: 
{
    "resourceType": "Bundle",
    "id": "bd768eef-2a79-404c-bf0c-89d3903b572e",
    "type": "searchset",
    "total": 27319,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=2"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=3"
        },
        {
            "relation": "previous",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=1"
        }
    ],

Number of entries: 10


In [53]:
# we can control the number of search results by passing the _count parameter
response = get(base + '/Patient?_count=1')
peek('Single resource per page: \n' + response.text, 25)
print('Number of entries: ' + str(len(response.json().get('entry'))))

Single resource per page: 
{
    "resourceType": "Bundle",
    "id": "5d6bb5e7-6db3-4a27-ab78-596cd0b41abb",
    "type": "searchset",
    "total": 27319,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
            "resource": {
                "resourceType": "Patient",
                "id": "17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
                "meta": {
                    "versionId": "1",
       

In [54]:
# if you're only interested in the count, you can specify that via either
# A. _count=0 (0 results per page); or
# B. _summary=count

print(get(base + '/Patient' + '?' + '_summary=count').text)

{
    "resourceType": "Bundle",
    "id": "8d82cfe3-1b10-4759-9589-67d59a411c12",
    "meta": {
        "tag": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
                "code": "SUBSETTED",
                "display": "subsetted"
            }
        ]
    },
    "type": "searchset",
    "total": 27319,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_summary=count&_page=1"
        }
    ]
}


In [55]:
# if you want a lot of results per page, you can reduce the amount of data returned via the _summary or _elements parameters

# https://www.hl7.org/fhir/search.html#summary
# look for the Σ flag in the Resource Content section of the resource page in the specification for what elements are considered "summary" elements 
reponse = get(base + '/Patient?_count=1' + '&' + '_summary=true')
print('Summary: \n' + str(response.json().get('entry')[0].get('resource').keys()))

Summary: 
dict_keys(['resourceType', 'id', 'meta', 'text', 'extension', 'identifier', 'name', 'telecom', 'gender', 'birthDate', 'address', 'maritalStatus', 'multipleBirthBoolean', 'communication'])


In [56]:
# need more control?
# you can use the _elements parameter to ask for specific fields back (although the server should include required fields and modifier fields as well)
response = get(base + '/Patient?_count=1' + '&' + '_elements=id,gender')
print('Elements: \n' + str(response.json().get('entry')[0].get('resource').keys()))

Elements: 
dict_keys(['resourceType', 'id', 'meta', 'gender'])


In [57]:
# this can add up!

response = get(base + '/Patient?_count=100')
print('Normal: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')

response = get(base + '/Patient?_count=100&_summary=true')
print('Summary: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')

response = get(base + '/Patient?_count=100&_elements=id,gender,birthDate')
print('Elements: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')


Normal: 	892360 bytes 	(1.934766 s)
Summary: 	516647 bytes 	(0.416664 s)
Elements: 	104260 bytes 	(0.348057 s)


In [58]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
response = get(base + '/Patient' + '?' + 'gender=male')
print('Response code: ' + str(response.status_code))
peek('Response body: \n' + response.text, 25)

Response code: 200
Response body: 
{
    "resourceType": "Bundle",
    "id": "6ed8b410-8aa6-4f61-8ac8-8a24cceb244b",
    "type": "searchset",
    "total": 12925,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&gender=male&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&gender=male&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
            "resource": {
                "resourceType": "Patient",
                "id": "17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
                "meta": {
           

In [59]:
print('male:   \t' + str(get(base + '/Patient' + '?' + 'gender=male' + '&' + '_summary=count').json().get('total')))
print('female: \t' + str(get(base + '/Patient' + '?' + 'gender=female' + '&' + '_summary=count').json().get('total')))

male:   	12925
female: 	14394


In [60]:
# use the "missing" modifier to look for resources that do NOT have a value for the target parameter
response = get(base + '/Patient' + '?' + 'gender:missing=true' + '&' + '_summary=count')
print('missing gender: ' + str(response.json().get('total')))

missing gender: 0


In [61]:
# search parameters have types

# gender is considered a "token" search parameter

# Token search
# this parameter type is common for 'coded' values (Code, Coding, and CodeableConcept) and identifiers
# token values consist of a system and a code, although sometimes the system is implicit (like in the case of gender)
# users can search on the system and code (system|code), the code alone (code), system-less codes (|code), or even the system alone (system|)
response = get(base + '/Patient' + '?' + 'gender=http://hl7.org/fhir/administrative-gender|male' + '&_count=1&_elements=gender')
peek('male:\n' + str(response.json()))


# there are also Number, Date/DateTime, String, Reference, Quantity, URI, and Composite parameter types

male:
{'resourceType': 'Bundle', 'id': 'bb6132a5-7147-49e0-8cd7-edd8d370fd36', 'meta': {'tag': [{'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationValue', 'code': 'SUBSETTED', 'display': 'subsetted'}]}, 'type': 'searchset', 'total': 12925, 'link': [{'relation': 'self', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&gender=http://hl7.org/fhir/administrative-gender%7Cmale&_elements=gender&_page=1'}, {'relation': 'next', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&gender=http://hl7.org/fhir/administrative-gender%7Cmale&_elements=gender&_page=2'}], 'entry': [{'fullUrl': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c', 'resource': {'resourceType': 'Patient', 'id': '17598bed122-d90e0186-8458-4482-a580-5b18b5

In [62]:
# String search
response = get(base + '/Patient' + '?' + 'family=Smith' + '&_elements=name')
print('Smiths:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
        

Smiths:
17598d8e9df-706d630f-784d-473d-a0d2-37af26e3c36e: Smitham
17598dc69c4-fe632367-ce4f-413b-9639-5f409631fdbd: Smith
17598e7faf2-37231ae2-90fa-4668-90cc-9d413e44ff73: Smitham
17598fe2d81-8588df6c-a4e5-486f-8de7-f2a457e1ecbb: Smitham
175991c2ff1-8b5aef81-6ed2-4173-8ee0-3a38612ece43: Smitham
175991de90e-b9364b28-a200-4d19-9bba-1cd4d5d9f037: Heller, Smith
175992f962b-159acf20-e5d5-4370-bf38-8e02eef6d761: Smith
17599326aa9-40e01d7e-1d5b-4f1b-9bed-9c0a0b031df1: Smith
175993420d0-eb6a0968-589a-4a71-a010-19e84056c44e: Smitham
17599375dc6-21078467-b048-499a-b8c4-9f5ddfc6ffd3: Haley, Smitham


In [63]:
# wait, "Smitham" !?

# string search performs a case-insensitive "begins-with" search by default!
# use the modifier ":exact" if you want exact matches (and improved performance)
response = get(base + '/Patient' + '?' + 'family:exact=Smith' + '&_elements=name')
print('Smiths:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
print()

# string search also has a ":contains" modifier
response = get(base + '/Patient' + '?' + 'family:contains=ski' + '&_elements=name')
print('Skis:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))


Smiths:
17598dc69c4-fe632367-ce4f-413b-9639-5f409631fdbd: Smith
175991de90e-b9364b28-a200-4d19-9bba-1cd4d5d9f037: Heller, Smith
175992f962b-159acf20-e5d5-4370-bf38-8e02eef6d761: Smith
17599326aa9-40e01d7e-1d5b-4f1b-9bed-9c0a0b031df1: Smith
175993c2874-5f898109-a18d-4d6e-8cfb-ecd69eb37862: Smith
17599522520-bdb4311f-3765-45f2-bd5d-1f1aca14a931: Smith
17599539a99-bccf02f3-57ef-4399-a4c6-0296aa8356c1: Purdy, Smith
1759954cc00-9def2bdc-a444-4e84-bf77-a0e79e449d19: Rosenbaum, Smith
175995a5564-869be93e-c7c2-4cab-9951-d15063f39975: Wintheiser, Smith
175995ad366-3722d3d9-c56e-4d06-ba32-1b662e18bfbe: Wehner, Smith

Skis:
17598bf6496-92b7682f-d882-4b3c-8e37-178b11cde674: Gusikowski, West
17598c069d0-3b1b68c5-13e0-4c46-b929-1fea6d13454f: Osinski
17598c0689e-e60c055e-c166-406d-a9b0-f6d113292dcc: Gutkowski
17598c10cbc-642eb489-8d83-4220-8591-a31adc7a9189: Gusikowski
17598c3ebc7-63c73ac0-7cc7-492f-b1f5-661c9da0b95e: Gulgowski
17598c45f73-e1ac3687-ae53-4591-b9f7-65deb78c8a1b: Jaskolski
17598c61295-b

In [64]:
# Date search
response = get(base + '/Patient' + '?' + 'birthdate=1984' + '&_elements=birthDate')
print('Born in 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

Born in 1984:
17598c01014-694ba51f-a02d-4686-b956-7d6eb9d7e5fa: 1984-12-05
17598c330b0-817d8bac-b275-449d-8163-0ae095d6a7d5: 1984-03-23
17598d78d31-639632e7-4f79-4b1a-81e8-f462de2c5912: 1984-09-18
17598dc812f-37fd8fe9-e900-4f11-b166-5515fd7b4125: 1984-12-26
17598dda658-9243965e-f6e2-4909-b2f6-89dcd7dcf799: 1984-05-04
17598de13a7-c2c48b25-a1cc-498a-9b98-08629619ae15: 1984-03-04
17598e32314-d67c7847-5ca3-4b0c-86df-195e40626a65: 1984-10-05
17598e54dfe-683137bf-6a96-4b94-9218-7be5ff4ee2fb: 1984-04-21
17598eed38d-eb24c019-4ab9-4f14-9c66-db95a1ed22dc: 1984-10-23
17598f52f36-17f4bfd8-c305-468f-9c38-792a461e28a3: 1984-04-26


In [65]:
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"
response = get(base + '/Patient' + '?' + 'birthdate=eb1984' + '&_elements=birthDate')
print('Born before 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

response = get(base + '/Patient' + '?' + 'birthdate=sa1984' + '&_elements=birthDate')
print('\n' + 'Born after 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

# some servers support ap(approximately equal) as well, although the spec lets the server decide exactly what that means...
response = get(base + '/Patient' + '?' + 'birthdate=ap1984' + '&_elements=birthDate')
print('\n' + 'Born "around" 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

Born before 1984:
17598bf0165-6eda45e2-f9c1-4c4c-9ef2-1776cfbb0a7d: 1976-10-11
17598bf1688-037fcb72-18af-40ce-a96b-c393a595dd49: 1975-07-10
17598bf28f3-257b9fe9-9084-41d6-bb0f-43b4729a7f0d: 1982-01-13
17598bf36c7-fedcedc3-b78c-4688-82ef-622e0cc71b22: 1973-03-03
17598bf6cd6-d8388338-897d-48a0-b334-a877459941b2: 1979-03-01
17598bf3d06-15fa6deb-59ad-4bd2-b8a1-8a2fa606d084: 1924-10-07
17598bf6496-92b7682f-d882-4b3c-8e37-178b11cde674: 1948-12-25
17598bf98b2-789821a7-b30c-48d4-b388-5343a933d8ad: 1972-06-15
17598bfb281-9ca10cb0-6ba0-4691-85f4-424423b1869d: 1970-12-02
17598bfa2da-9c19d3ed-5bef-49ab-b6b0-1d731446ddf6: 1960-11-21

Born after 1984:
17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c: 1990-02-05
17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f: 1986-07-15
17598bf4a5d-185e291b-d0e3-42d3-9b70-eeae60debeec: 2005-11-09
17598bf2ea9-7238cb29-0108-47a7-bf50-c94c6468ad2b: 2020-01-19
17598bf555d-8e5857a0-4da9-4e6c-b9be-8624f80cf080: 1990-01-07
17598bf809c-1a099ca4-cdb9-4837-8692-51338082cd97:

In [66]:
# Reference search
response = get(base + '/Patient?general-practitioner:missing=false&_elements=generalPractitioner,link,managingOrganization&_count=1')
peek('Patients with a general-practitioner:   \n' + str(response.json()))

# since our model doesn't have any reference fields on the Patient resources, lets look at Observations instead
response = get(base + '/Condition' + '?' + 'subject=Patient/17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('Conditions for patient 17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('code'))

# when the type of the reference is fixed to a single value, it can be omitted (Patient/x -> x)
response2 = get(base + '/Condition' + '?' + 'patient=17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response2.json().get('entry')))

# a reference to a resource's full url on the server should be equivalent to the relative reference format mentioned above
response3 = get(base + '/Condition' + '?' + 'patient=' + base + '/Patient/17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response3.json().get('entry')))

# references can also reference resources on other servers

Patients with a general-practitioner:   
{'resourceType': 'Bundle', 'id': 'f9a06eae-344c-44cf-8d27-a3fa3bb401bb', 'meta': {'tag': [{'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationValue', 'code': 'SUBSETTED', 'display': 'subsetted'}]}, 'type': 'searchset', 'total': 0, 'link': [{'relation': 'self', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&general-practitioner:missing=false&_elements=generalPractitioner,link,managingOrganization&_page=1'}]}

Conditions for patient 17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f:
{'coding': [{'system': 'http://snomed.info/sct', 'code': '10509002', 'display': 'Acute bronchitis (disorder)'}], 'text': 'Acute bronchitis (disorder)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '195662009', 'display': 'Acute viral pharyngitis (disorder)'}], 'text': 'Acute viral pharyngitis (disorder)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '68235

In [38]:
# Chaining

# where reference parameters get really interesting is when you want to query one resource type based on a property of another resource to which its linked
# for example, here is a search for Type II Diabetes in female patients
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&' + 'patient:Patient.gender=female' + '&_count=1')
peek('Type II Diabetes in female patients:   \n' + str(response.json()))
    
# for example, here is a search for blood pressure values for female patients
# TODO: fix this query!
#response = get(base + '/Observation' + '?' + 'code=http://loinc.org|85354-9' + '&' + 'date=gt2020-01-01' + '&' + 'patient:Patient.gender=female' + '&_count=1')
#peek('Blood Pressure Observations for female patients since 2020-01-01:   \n' + str(response.json()))

Type II Diabetes in female patients:   
{'resourceType': 'Bundle', 'id': '9a62b679-ae9a-4118-bc3c-e4c8961915ff', 'type': 'searchset', 'total': 468, 'link': [{'relation': 'self', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Condition?_count=1&code=http://snomed.info/sct%7C44054006&patient:Patient.gender=female&_page=1'}, {'relation': 'next', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Condition?_count=1&code=http://snomed.info/sct%7C44054006&patient:Patient.gender=female&_page=2'}], 'entry': [{'fullUrl': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Condition/17598ca24de-8a64595d-1f7f-4a9d-af9b-9623bbd3db6d', 'resource': {'resourceType': 'Condition', 'id': '17598ca24de-8a64595d-1f7f-4a9d-af9b-9623bbd3db6d', 'meta': {'versionId': '1', 'lastUpdated': '2020-11-05T14:23:11.963Z', 'profile': ['http://hl7.or

In [42]:
# Reverse chaining

# references can be search the other way around via the "_has" parameter
response = get(base + '/Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006' + '&_count=1')
peek('Patients with Type II Diabetes:   \n' + str(response.json()))

# TODO: fix this query!
#response = get(base + '/Patient?gender=female' + '&' + '_has:Observation:patient:code=http://loinc.org|85354-9' + '&_count=1')
#peek('Female patients with Blood Pressure Observations:   \n' + str(response.json()))

Patients with Type II Diabetes:   
{'resourceType': 'Bundle', 'id': '1a9049db-1fd1-45d1-ac2e-674a7b088fdb', 'type': 'searchset', 'total': 823, 'link': [{'relation': 'self', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_has:Condition:patient:code=http://snomed.info/sct%7C44054006&_page=1'}, {'relation': 'next', 'url': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_has:Condition:patient:code=http://snomed.info/sct%7C44054006&_page=2'}], 'entry': [{'fullUrl': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598c0ea1f-f838725e-91e0-4a98-9eb6-f3e7323e9dfa', 'resource': {'resourceType': 'Patient', 'id': '17598c0ea1f-f838725e-91e0-4a98-9eb6-f3e7323e9dfa', 'meta': {'versionId': '1', 'lastUpdated': '2020-11-05T14:13:01.093Z', 'profile': ['http://hl7.org/fhir/us/core/StructureDef

In [43]:
# Includes

# its also possible to get a resource and its related resources back in a single query
response = get(base + '/Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient' + '&_count=2')
peek('Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))


Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:

Total: 823
Condition: 17598c0ea20-7b251c64-b10a-4942-b72b-3014af79875f
Condition: 17598c6d269-b8bd24cc-0280-48f8-a835-db262812387e
Patient: 17598c0ea1f-f838725e-91e0-4a98-9eb6-f3e7323e9dfa
Patient: 17598c6d269-6c393947-fbeb-44b7-9ec7-fbaede13bc8a


In [44]:
# Reverse Includes

response = get(base + '/Patient?gender=female' + '&' + '_revinclude=Condition:patient' + '&_count=2')
peek('Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))

Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:

Total: 14394
Patient: 17598bf36c7-fedcedc3-b78c-4688-82ef-622e0cc71b22
Patient: 17598bf4a5d-185e291b-d0e3-42d3-9b70-eeae60debeec
Condition: 17598bf4a5e-b9e99052-25a6-4400-a8b8-da48b8cbb22d
Condition: 17598bf4a65-ee99ca87-65c3-4a14-900f-02e90ec4db4a
Condition: 17598bf4a66-49d7afde-a7f1-42e4-858d-990f49a2f1ca
Condition: 17598bf4a66-39d8080e-27ee-488d-9e35-59275a486457
Condition: 17598bf4a6f-b87ac7b6-6faf-49b5-95fd-3eddf4f7494c
Condition: 17598bf4a6f-ec9abd6f-ee59-4f71-8fa5-c737ac1be63b
Condition: 17598bf4a70-b98f87d7-d469-4b3d-987e-02b78336b82e
Condition: 17598bf4a70-6e7537b4-1a90-4a3c-842c-6c5c1db45ad3
Condition: 17598bf4a70-ca048f40-02a3-4c24-a1b6-f53ffb3c1667
Condition: 17598bf36c8-5800bd67-4c08-4a68-8968-054c670ef2a6
Condition: 17598bf36c8-7d2861ad-e607-4122-90b3-74265240aaca
Condition: 17598bf36c9-d53628c0-8fc4-4570-bb9a-25af21328712
Condition: 17598bf36c9-9dec87f6-1a92-4b8

## Putting it together

In [45]:
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&_count=1')
print('Patients with Type II Diabetes:  ' + str(response.json().get('total')))

Patients with Type II Diabetes:  823


In [46]:
# SNOMED concepts for comorbidities of Type II Diabetes
#coronary heart disease (CHD), 53741008
#chronic kidney disease (CKD), 709044004
#atrial fibrillation, 49436004
#stroke, 230690007
#hypertension, 38341003
#heart failure, 84114007
#peripheral vascular disease (PVD), 400047006
#rheumatoid arthritis, 69896004
#Malignant neoplasm, primary (morphologic abnormality), 86049000
#Malignant neoplastic disease (disorder), 363346000
#osteoporosis, 64859006
#depression, 35489007
#asthma, 195967001
#chronic obstructive pulmonary disease (COPD), 13645005
#dementia, 52448006
#severe mental illness (SMI), 391193001
#epilepsy, 84757009
#hypothyroidism, 40930008
#learning disability, 1855002

print('CHD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '53741008').json().get('total')))
print('CKD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '709044004').json().get('total')))
print('AFib: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '49436004').json().get('total')))
print('stroke: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '230690007').json().get('total')))
print('hypertension: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '38341003').json().get('total')))
print('heart failure: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84114007').json().get('total')))
print('PVD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '400047006').json().get('total')))
print('arthritis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '69896004').json().get('total')))
print('cancer: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '86049000').json().get('total') + get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '363346000').json().get('total')))
print('osteoporosis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '64859006').json().get('total')))
print('depression: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '35489007').json().get('total')))
print('asthma: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '195967001').json().get('total')))
print('COPD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '13645005').json().get('total')))
print('dementia: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '52448006').json().get('total')))
print('SMI: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '391193001').json().get('total')))
print('epilepsy: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84757009').json().get('total')))
print('hypothyroidism: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '40930008').json().get('total')))
print('learning disability: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '1855002').json().get('total')))

CHD: 			848
CKD: 			0
AFib: 			270
stroke: 		830
hypertension: 		0
heart failure: 		54
PVD: 			0
arthritis: 		38
cancer: 		0
osteoporosis: 		634
depression: 		0
asthma: 		19
COPD: 			0
dementia: 		0
SMI: 			0
epilepsy: 		455
hypothyroidism: 	0
learning disability: 	0


In [47]:
# Patients with Type II Diabetes *and* comorbidities
hasDiabetes = base + '/Patient?_elements=id&_has:Condition:patient:code=http://snomed.info/sct|44054006'

def printPatientsWithComorbidity(conceptId):
    responseJSON = get(hasDiabetes + '&_has:Condition:patient:code=http://snomed.info/sct|' + conceptId).json()
    print('Total: ' + str(responseJSON.get('total')))
    if 'entry' in responseJSON:
        for entry in responseJSON.get('entry'):
            print(entry.get('resource').get('id'), end=", ")
    print('\n')

print('CHD:')
printPatientsWithComorbidity('53741008')

print('AFib:')
printPatientsWithComorbidity('49436004')

print('stroke:')
printPatientsWithComorbidity('230690007')

print('heart failure:')
printPatientsWithComorbidity('84114007')

print('arthritis:')
printPatientsWithComorbidity('69896004')

print('osteoporosis:')
printPatientsWithComorbidity('64859006')

print('asthma:')
printPatientsWithComorbidity('195967001')

print('epilepsy:')
printPatientsWithComorbidity('84757009')

CHD:
Total: 13
1759a06f11a-77b7ee76-ae8d-47a8-9a8e-17e7278e4137, 1759bccd5dc-a4192660-b224-4d88-acf6-bfb538fc0052, 1759c042626-da786c63-fcd4-4a86-aaec-54ec28458861, 1759c7179ab-cdc927fb-ec72-4c10-961e-a424bb8241f3, 1759c906bce-6fcc151d-240e-441f-aa3f-3a8123222d0c, 1759c9d0e32-10946dda-2ac9-4f8e-a6ba-eade9aba6ac8, 175b90993dc-7568b0d3-91ce-4862-be00-7cb99389f323, 175ba73ffc5-86182577-c52f-4c24-b09c-acfdc0043b8c, 175babfca99-31bcaafc-876a-453a-905c-b0482607b2c5, 175bb745e4f-1f3f83a8-6f6c-43da-adf1-fdc5440d88b5, 
AFib:
Total: 15
1759b62a9bc-650fc722-8bb1-4a81-827c-95b73a83d7b4, 1759be696c1-fd6cfa13-8e69-454f-ad73-d3b27a051f6f, 1759c1d61ec-500b2890-fcc9-428c-86c2-bcb15a5cbaa5, 1759c4dfa60-d50fbfcb-dda0-4669-82c8-42e73c5bb239, 1759c76bdcd-dfc226a9-37b6-4a72-b303-1613ed7ec838, 175b9f4c2a0-e971ee19-f55e-4319-85bd-57737b23ba26, 175ba948672-bc9ba458-81df-4def-8064-da02406e9f62, 175ba9518ee-29c6291a-ae29-4c62-8283-6c5a1264c404, 175ba9ca813-0a7f0565-f701-4c66-9d11-7b2754dddf6d, 175baa492a2-c73064

In [48]:
peek('A1c:' + str(get(base + '/Observation?_elements=code&code=http://loinc.org|' + '4548-4').text), 50)


A1c:{
    "resourceType": "Bundle",
    "id": "368e11d8-d68f-4b0c-a3b1-fc52db8bc31f",
    "meta": {
        "tag": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
                "code": "SUBSETTED",
                "display": "subsetted"
            }
        ]
    },
    "type": "searchset",
    "total": 55867,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Observation?_count=10&code=http://loinc.org%7C4548-4&_elements=code&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Observation?_count=10&code=http://loinc.org%7C4548-4&_elements=code&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-

## Bulk Export

## Using the fhirpy pip module

FHIR has a rich library of open source clients and servers for almost every lanuage.
For python, there is a module called `fhirpy` which can be found at https://pypi.org/project/fhirpy/

In [ ]:
!pip install fhirpy

In [ ]:
from fhirpy import SyncFHIRClient

fhir = SyncFHIRClient(base)

In [ ]:
searchSet = fhir.resources('Practitioner')
searchSet.fetch()